In [1]:
import requests # 웹 페이지 소스를 얻기 위한 패키지(기본 내장 패키지이다.)
from bs4 import BeautifulSoup # 웹 페이지 소스를 얻기 위한 패키지, 더 간단히 얻을 수 있다는 장점이 있다고 한다.
from datetime import datetime                                # (!pip install beautifulsoup4 으로 다운받을 수 있다.)
import pandas as pd # 데이터를 처리하기 위한 가장 기본적인 패키지
import time # 사이트를 불러올 때, 작업 지연시간을 지정해주기 위한 패키지이다. (사이트가 늦게 켜지면 에러가 발생하기 때문)
import urllib.request #
from selenium.webdriver import Chrome
import json
import re     

In [2]:
import time

In [3]:
start_url = 'https://www.youtube.com/channel/UC9kmlDcqksaOnCkC_qzGacA/featured'
#videos

In [4]:
delay = 3

In [5]:
browser = Chrome()
browser.implicitly_wait(delay)

In [6]:
browser.get(start_url)  

In [7]:
browser.find_element_by_xpath('//*[@class="style-scope ytd-c4-tabbed-header-renderer"]/div[text()="동영상"]').click()
#현재 text로 클릭이 안되는 이유는 text의 공백부분이 너무 많이 때문에 클릭하기 위해선 밑의 코드를 따른다

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//*[@class="style-scope ytd-c4-tabbed-header-renderer"]/div[text()="동영상"]"}
  (Session info: chrome=73.0.3683.86)
  (Driver info: chromedriver=2.38.552522 (437e6fbedfa8762dec75e2c5b3ddb86763dc9dcb),platform=Windows NT 10.0.17134 x86_64)


In [8]:
browser.find_element_by_xpath('//*[@class="scrollable style-scope paper-tabs"]/paper-tab[2]').click()

In [9]:
body = browser.find_element_by_tag_name('body')

In [10]:
body

<selenium.webdriver.remote.webelement.WebElement (session="22afc8ce6ebf61994e775527ba0115da", element="0.7494078889596965-2")>

In [11]:
from selenium.webdriver.common.keys import Keys


In [12]:
num_of_pagedowns = 20
#10번 밑으로 내리는 것
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [13]:
html0 = browser.page_source

In [14]:
html = BeautifulSoup(html0,'html.parser')

In [15]:
video_list0 = html.find('div',{'id':'items'})

In [16]:
video_list2 = video_list0.find_all('ytd-grid-video-renderer',{'class':'style-scope ytd-grid-renderer'})

In [18]:
base_url = 'http://www.youtube.com'

In [19]:
isabe_url = []
for i in range(len(video_list2)):
    url = base_url+video_list2[i].find('a',{'id':'thumbnail'})['href']
    isabe_url.append(url)

In [169]:
isabe = pd.DataFrame({'name':[],
                     'thumbnail':[],
                     'view':[],
                     'previous_time':[],
                     'video_url':[],
                     'start_date':[],
                     'comment':[],
                     'likes_num':[],
                     'unlikes_num':[]})
base_url = 'http://www.youtube.com'
delay = 3
browser = Chrome()
browser.implicitly_wait(delay)
browser.maximize_window()

for i in range(0,20):
    name = video_list2[i].find('a',{'id':'video-title'}).text
    
    thum = video_list2[i].find('a',{'id':'thumbnail'}).find('img')['src']
    
    url = base_url+video_list2[i].find('a',{'id':'thumbnail'})['href']
        
    
    meta0 =video_list2[i].find('div',{'id':'metadata-line'})
    
    view = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[0].text.split()[1]
    #조회수는 숫자만 뽑을 수 있고 모든 데이터는 원하는 형식으로 뽑을 수 있음
    previous = meta0.find_all('span',{'class':'style-scope ytd-grid-video-renderer'})[1].text
    
    start_url = isabe_url[i]
    browser.get(start_url) 
    body = browser.find_element_by_tag_name('body')
    
    time.sleep(1.5)#브라우저 로딩시간기다려야함
    
    num_of_pagedowns = 4 #page_down은 2로 놔도 괜찮을 듯 최대화 했으므로
    while num_of_pagedowns:
        body.send_keys(Keys.PAGE_DOWN)
        time.sleep(1.5) #스크롤 속도를 지정할 수 있는 것인데  1.5초가 가장 적당할 듯 1초면 너무 빠름
        #현재는 스크롤을 7번만 하도록 설정되어 있으나 이후에 댓글을 크롤링하기 위해서는 스크롤을 댓글이 안보
        #일때까지 내리면 될텐데 시간이 엄청 오래 걸릴 것임 - 따라서 적당한 수준의 댓글을 크롤링하는것이 좋음
        num_of_pagedowns -= 1
    #여기에 time.sleep를 집어 넣는 방법을 고안
    html0 = browser.page_source
    html = BeautifulSoup(html0,'html.parser')
    
    start_date = html.find('span',{'class':'date style-scope ytd-video-secondary-info-renderer'}).text
    comment = html.find('h2',{'id':'count'}).find('yt-formatted-string').text
    
    #좋아요수
    likes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('좋아요')}).text+'개'

    #싫어요수
    unlikes_num = html.find('yt-formatted-string',{'id':'text','class':'style-scope ytd-toggle-button-renderer style-text','aria-label':re.compile('싫어요')}).text+'개'

    insert_data = pd.DataFrame({'name':[name],
                     'thumbnail':[thum],
                     'view':[view],
                     'previous_time':[previous],
                     'video_url':[url],
                     'start_date':[start_date],
                     'comment':[comment],
                     'likes_num':[likes_num],
                     'unlikes_num':[unlikes_num]})
    
    isabe = isabe.append(insert_data)
isabe.index = range(len(isabe))

In [170]:
isabe

,name,thumbnail,view,previous_time,video_url,start_date,comment,likes_num,unlikes_num
0,[Eng] 🛫해외 갈 때 꼭! 챙기는 필수템 (출장&여행) l 이사배(RISABAE...,https://i.ytimg.com/vi/vR8NPTKi7t4/hqdefault.j...,11만회,1일 전,http://www.youtube.com/watch?v=vR8NPTKi7t4,게시일: 2019. 4. 8.,댓글 182개,2.5천개,44개
1,[Eng] 흑백 메이크업 (feat.아리아나그란데) Black&White Makeu...,https://i.ytimg.com/vi/Hhowj4f1geQ/hqdefault.j...,14만회,1주 전,http://www.youtube.com/watch?v=Hhowj4f1geQ,게시일: 2019. 4. 1.,댓글 326개,3.1천개,39개
2,[Eng] 화사한 코랄 메이크업💐[하객룩+데이트에도 딱💕] l 이사배(RISABAE...,https://i.ytimg.com/vi/NRGosJR4zp0/hqdefault.j...,34만회,1주 전,http://www.youtube.com/watch?v=NRGosJR4zp0,게시일: 2019. 3. 27.,댓글 411개,5.8천개,63개
3,[Eng] 배우 수현 커버 메이크업💫 CLAUDIA KIM COVER MAKEUP ...,https://i.ytimg.com/vi/698XKr0cYyM/hqdefault.j...,22만회,2주 전,http://www.youtube.com/watch?v=698XKr0cYyM,게시일: 2019. 3. 22.,댓글 487개,5.3천개,57개
4,[GRWM] 미세먼지 심한 날 메이크업 + 붙임머리/염색 vlog💇🏻‍♀💇🏼‍♀✨...,https://i.ytimg.com/vi/pEtwG60WpdI/hqdefault.j...,30만회,3주 전,http://www.youtube.com/watch?v=pEtwG60WpdI,게시일: 2019. 3. 19.,댓글 327개,5.1천개,82개
5,백화점 쇼핑하울 속옷부터-가방까지!👙👛🥰 [+엄마 옷] l 이사배(RISABAE M...,https://i.ytimg.com/vi/p26ggSim0aY/hqdefault.j...,44만회,3주 전,http://www.youtube.com/watch?v=p26ggSim0aY,게시일: 2019. 3. 15.,댓글 386개,5.6천개,148개
6,[Eng] 봄이 오기전에 봄 메이크업 해봄🌼 Spring Makeup Tutoria...,https://i.ytimg.com/vi/ZytjDm5lnkw/hqdefault.j...,42만회,1개월 전,http://www.youtube.com/watch?v=ZytjDm5lnkw,게시일: 2019. 3. 8.,댓글 589개,7.9천개,136개
7,[Eng] 백투베이직-블러셔편 : 나에게 맞는 블러셔 정복하기 ! l 이사배(RIS...,https://i.ytimg.com/vi/0F0aoPTL2mc/hqdefault.j...,55만회,1개월 전,http://www.youtube.com/watch?v=0F0aoPTL2mc,게시일: 2019. 2. 26.,댓글 846개,1.1만개,108개
8,[GRWM] 어떤 컬러 의상에도 잘 어울리는 메이크업💙같이 준비해요!!✨ l 이사배...,https://i.ytimg.com/vi/WiKDflV6BPY/hqdefault.j...,43만회,1개월 전,http://www.youtube.com/watch?v=WiKDflV6BPY,게시일: 2019. 2. 21.,댓글 602개,8.2천개,97개
9,[Eng] 활용도 10000% 면접메이크업 l 이사배(RISABAE Makeup),https://i.ytimg.com/vi/lB-WHUhHVzw/hqdefault.j...,35만회,1개월 전,http://www.youtube.com/watch?v=lB-WHUhHVzw,게시일: 2019. 2. 15.,댓글 561개,7.8천개,89개


In [30]:
isabe

,name,thumbnail,view,previous_time,video_url,start_date,comment
0,1999st 메이크업하고 힙합우상★타이거JK님★만나기!! + 녹음까지🎧 l이사배(R...,https://i.ytimg.com/vi/3X51q_VEaNA/hqdefault.j...,24만회,2개월 전,http://www.youtube.com/watch?v=3X51q_VEaNA,게시일: 2019. 1. 12.,댓글 835개
1,[Eng] 민낯에 레드립st 메이크업💄 (레드립 꿀팁!) + 이벤트(종료!) l 이...,https://i.ytimg.com/vi/t3p15Qzirjo/hqdefault.j...,74만회,2개월 전,http://www.youtube.com/watch?v=t3p15Qzirjo,게시일: 2019. 1. 11.,"댓글 1,291개"
2,[GRWM] 필카느낌 셀카잘나오는 메이크업📸😏✌🏻같이준비해요! l 이사배(RISAB...,https://i.ytimg.com/vi/y0zQgUWpzHk/hqdefault.j...,30만회,3개월 전,http://www.youtube.com/watch?v=y0zQgUWpzHk,게시일: 2019. 1. 4.,댓글 313개
3,2018 잘~썼다! 아이템 TOP 9 🎉 l 이사배(RISABAE Makeup),https://i.ytimg.com/vi/P0-gKMq48aI/hqdefault.j...,98만회,3개월 전,http://www.youtube.com/watch?v=P0-gKMq48aI,게시일: 2018. 12. 29.,댓글 746개
4,[Eng] 연말파티에 딱! 화려한-브라운메이크업🥂✨ l 이사배(RISABAE Mak...,https://i.ytimg.com/vi/hQ2ffSZuAPw/hqdefault.j...,20만회,3개월 전,http://www.youtube.com/watch?v=hQ2ffSZuAPw,게시일: 2018. 12. 24.,댓글 229개


In [178]:
start_url = isabe_url[1]


In [90]:
delay = 3
browser = Chrome()
browser.implicitly_wait(delay)

In [180]:
browser.get(start_url)  

In [181]:
body = browser.find_element_by_tag_name('body')

In [134]:
num_of_pagedowns = 20
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(2)
    num_of_pagedowns -= 1

In [182]:
num_of_pagedowns = 7
while num_of_pagedowns:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1)
    num_of_pagedowns -= 1

In [183]:
html0 = browser.page_source

In [184]:
html = BeautifulSoup(html0,'html.parser')

In [185]:
html.find('span',{'class':'date style-scope ytd-video-secondary-info-renderer'}).text

'게시일: 2019. 4. 1.'

In [186]:
html.find('h2',{'id':'count'}).find('yt-formatted-string').text

'댓글 324개'

# 유튜브 댓글 크롤링

In [22]:
from selenium.webdriver.chrome.options import Options

In [23]:
options = Options()

In [27]:
options.add_argument('--start-maximized') #--start-fullscreen이있다. 하지만 최대화가 아니라 전체화면임

In [30]:
browser = Chrome()#chrome_options=options
browser.implicitly_wait(delay)
browser.maximize_window()#전체화면이 아니라 최대화하는 방법

In [35]:
start_url = isabe_url[0]

In [82]:
browser.get(start_url) 
body = browser.find_element_by_tag_name('body')
#웹드라이버로 url접속한 뒤 스크롤 내리기 위한 소스를 받아오는 것

In [83]:
num_page_down = 1
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1
        

In [84]:
#이 세개가 다 되는 코드임 - 자동제어 창을 띄어놔야 코드실행이 빨라짐
#browser.find_element_by_xpath('//*[@id="sort-menu"]').click()
browser.find_element_by_xpath('//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()
#browser.find_element_by_xpath('//paper-menu-button[@class="style-scope yt-dropdown-menu"]').click()
#반드시 클릭은 해당 클릭할 화면이 보여야 클릭이 가능하다 당연히 제어하는 것이기 때문에

In [85]:
browser.find_element_by_xpath('//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()
#driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click()
#이렇게 직접 텍스트를 지정해줌으로써 클릭하게 할 수 있음 -> 현재는 인기댓글순임

In [86]:
num_page_down = 20
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1

In [87]:
html_s0 = browser.page_source

In [89]:
comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})

In [146]:
#댓글
comment0[5].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text

'필수템이라고 해서 추천해주시는 아이팔레트 기대했는데ㅠ따흐흑\ufeff'

#뷰티에 관심있는 집단 / 단순히 크리에이터를 좋아하는 분류 / 리뷰하는 아이템에 관심있는 사람 ....
#여러 군집으로 나눠서 텍스트마이닝도 가능할 듯

In [109]:
try:
    aa = comment0[0].find('span',{'id':'vote-count-left'}).text
    #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
    #re.findall('[0-9]',aa)
    #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
    like_num = "".join(re.findall('[0-9]',aa))
except:
    like_num = 0

In [140]:
bb = comment0[0].find('a',{'id':'author-text'}).find('span').text

In [144]:
youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))

### 이처럼 댓글을 크롤링을 진행하되 영상별로 댓글이 긁어져 오는 것이다
### 따라서 영상별 url을 반복문으로 짠 뒤 영상 name을 key로 댓글을 긁어온다.

In [156]:
comment_data = pd.DataFrame({'youtube_id':[],
                            'comment':[],
                            'like_num':[]})
browser = Chrome()#chrome_options=options
browser.implicitly_wait(delay)
browser.maximize_window()#전체화면이 아니라 최대화하는 방법

start_url = isabe_url[0]
browser.get(start_url) 
body = browser.find_element_by_tag_name('body')
#웹드라이버로 url접속한 뒤 스크롤 내리기 위한 소스를 받아오는 것

time.sleep(2)

num_page_down = 1
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1

#이 세개가 다 되는 코드임 - 자동제어 창을 띄어놔야 코드실행이 빨라짐
#browser.find_element_by_xpath('//*[@id="sort-menu"]').click()
browser.find_element_by_xpath('//paper-button[@class="dropdown-trigger style-scope yt-dropdown-menu"]').click()
#browser.find_element_by_xpath('//paper-menu-button[@class="style-scope yt-dropdown-menu"]').click()
#반드시 클릭은 해당 클릭할 화면이 보여야 클릭이 가능하다 당연히 제어하는 것이기 때문에

time.sleep(1.5)
browser.find_element_by_xpath('//paper-listbox[@class="dropdown-content style-scope yt-dropdown-menu"]/a[1]').click()
#driver.find_element_by_xpath('//*[@id="menu"]/a[2]/paper-item/paper-item-body/div[text()="최근 날짜순"]').click()
#이렇게 직접 텍스트를 지정해줌으로써 클릭하게 할 수 있음 -> 현재는 인기댓글순임

num_page_down = 20
while num_page_down:
    body.send_keys(Keys.PAGE_DOWN)
    time.sleep(1.5)
    num_page_down -= 1
    
html_s0 = browser.page_source
html_s = BeautifulSoup(html_s0,'html.parser')

comment0 = html_s.find_all('ytd-comment-renderer',{'class':'style-scope ytd-comment-thread-renderer'})

for i in range(len(comment0)):
    #댓글
    comment = comment0[i].find('yt-formatted-string',{'id':'content-text','class':'style-scope ytd-comment-renderer'}).text
    
    try:
        aa = comment0[i].find('span',{'id':'vote-count-left'}).text
        #정규표현식으로 숫자만 추출하는 것은 정규표현식에 대한 공부를 더 한 뒤 해결
        #re.findall('[0-9]',aa)
        #"".join(re.findall('[0-9]',aa)) -> 리스트 내부의 문자열의 합
        like_num = "".join(re.findall('[0-9]',aa)) + "개"
    except:
        like_num = 0
        
    bb = comment0[i].find('a',{'id':'author-text'}).find('span').text
    youtube_id = "".join(re.findall('[가-힣0-9a-zA-Z]',bb))
    
    insert_data =  pd.DataFrame({'youtube_id':[youtube_id],
                                               'comment':[comment],
                                               'like_num':[like_num]})

    comment_data = comment_data.append(insert_data)
    
comment_data.index = range(len(comment_data))





In [157]:
comment_data

,youtube_id,comment,like_num
0,l주비쥬jubijoux,패션테이프!! 신기하네용🤩 사배님 너무 멋있어요...자기관리하는 여자... 이시대의...,41개
1,오그린,언니가 먹는 영양제나 건강식품 추천영상도 보고싶어요!!ㅋㅋ 요즘 몸이 안좋아져서 건...,349개
2,지효,언니 분위기 완전 세뇨리rrrrerrrr따하고 엘lllllll레강스하고 그렇습니다....,90개
3,양은진,엇 그 1번 귀걸이네욧!! 물건사실 때 항상 분명한 이유가 있으신거같아용 충동구매로...,59개
4,낭만,저도 필수템으로 데려가시면 안돼여?﻿,296개
5,JKim,필수템이라고 해서 추천해주시는 아이팔레트 기대했는데ㅠ따흐흑﻿,3개
6,최채연,너무화려해....세상 연예인인데요 그냥ㅠㅠㅠ﻿,171개
7,HaNguyen,Can you do a tutorial for your makeup look in ...,4개
8,karnoroi,팥이 성질이 찬 식품이라서 손발이 차가우신 분들 드실때 조심하세요!\n사배님도 여성...,48개
9,jujin,언니는 어쩜..... 저 옷은 언니를 위해 만들어졌네여..ㅠㅠㅠㅠ💕 언니가 제 멘...,17개
